In [27]:
# imports
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium. webdriver. chrome. service import Service
from selenium. webdriver. chrome. options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from tqdm import tqdm
import pickle
from time import sleep

In [28]:
# 入力ファイル
INPUT_FILE = "shop_url_list_by_district/shop_url_list_kumamoto.pkl"
# 出力ファイル
OUTPUT_FILE = "shop_list_by_district/kumamoto_shop_list.csv"

In [29]:
# shop_url_listを読み込む
with open(INPUT_FILE, 'rb') as f:
    shop_url_list = pickle.load(f)
print(len(shop_url_list))
print(len(set(shop_url_list)))

205
205


In [30]:
# ブラウザ起動
options = Options ()
options.binary_location = "/usr/bin/chromium-browser" # Chromium のパス
# options.add_argument ("--headless")
options.add_argument ("--no-sandbox")
options.add_argument ("--disable-dev-shm-usage")
service = Service ("/usr/bin/chromedriver") # chromedriverのパス

driver = webdriver.Chrome(service=service, options=options)

In [31]:
# スクレイピング
shop_list = []
for shop_url in tqdm(shop_url_list):
    driver.get(shop_url)
    try:
        driver.find_element(By.LINK_TEXT, "電話番号を表示する").click()
        sleep(2)
    except:
        pass
    table = driver.find_element(By.ID, "Company-table-main")
    ths = table.find_elements(By.TAG_NAME, "th")
    th_texts = [th.text for th in ths]
    tds = table.find_elements(By.TAG_NAME, "td")
    td_texts = [td.text for td in tds]

    # 必要な情報のインデックスを取得
    name_index = th_texts.index('店舗名')
    try:
        phone_index = th_texts.index('電話番号')
    except:
        phone_index = None
    address_index = th_texts.index('住所')
    try:
        website_index = th_texts.index('ホームページ')
    except:
        website_index = None

    # 各情報を取得
    shop_name = td_texts[name_index]
    if phone_index is not None:
        phone_number = td_texts[phone_index].split('\n')[0]
    else:
        phone_number = ' - '
    address = td_texts[address_index]
    if website_index is not None:
        website = td_texts[website_index]
    else:
        website = ' - '

    dic={}
    dic['店舗名'] = shop_name
    dic['電話番号'] = phone_number
    dic['住所'] = address
    dic['ホームページ'] = website
    shop_list.append(dic)
driver.quit()

100%|██████████| 205/205 [09:15<00:00,  2.71s/it]


In [32]:
df = pd.DataFrame(shop_list)
print(df.shape)
print(df.duplicated().sum())
# 重複を削除
df = df.drop_duplicates()
print(df.shape)
print(df['店舗名'].nunique())
print(df['電話番号'].nunique() - 1)
print(df['住所'].nunique())
print(df['ホームページ'].nunique() - 1)

(205, 4)
0
(205, 4)
205
137
203
60


In [33]:
df.to_csv(OUTPUT_FILE, index=False)